In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import time
import json

options = Options()
options.headless = False

driver = webdriver.Chrome(options=options,service=ChromeService(ChromeDriverManager().install()))

In [4]:
def get_match_events(driver, whoscored_match_id=1640761):
    whoscored_match_url = f'https://www.whoscored.com/Matches/{whoscored_match_id}/Live/'
    driver.get(whoscored_match_url)
    scriptdiv = driver.find_elements(By.CSS_SELECTOR, 'script')
    # indicator_string = 'require.config.params["args"]'
    indicator_string = 'matchCentreData'
    for i in range(len(scriptdiv)):
        jstring = scriptdiv[-1*(i+1)].get_attribute('innerHTML')
        if indicator_string in jstring:            
            break
    jstring = jstring.replace('require.config.params["args"] = ', '')
    jstring = jstring.replace('\n', '')
    jstring = jstring.replace('            ', '')
    jstring = jstring.replace('{matchId:', '{"matchId":')
    jstring = jstring.replace('matchCentreData:', '"matchCentreData":')
    jstring = jstring.replace(
        'formationIdNameMappings', '"formationIdNameMappings"')
    jstring = jstring.replace(
        'matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    jstring = jstring.replace('};', '}')    
    
    d = json.loads(jstring)
    matchData = d['matchCentreData']
    matchId = d['matchId']
    events = matchData['events']
    player_dict = d['matchCentreData']['playerIdNameDictionary']
    players = {
        'id': player_dict.keys(),
        'name': player_dict.values()
    }
    players_df = pd.DataFrame(players).reset_index()
    players_df['id'] = players_df['id'].astype(int)
    event_list = []
    for e in enumerate(events):
        ev = e[1]
        try:
            player_id = ev['playerId']
        except:
            player_id = None
        try:
            player_name = players_df[players_df['id']
                                     == player_id]['name'].iloc[0]
            # print(f'{player_id} : {player_name}')
        except:
            player_name = None
        try:
            endX = ev['endX']
        except:
            endX = None
        try:
            endY = ev['endY']
        except:
            endY = None

        event_list.append({
            'id': ev['id'],
            'eventId': ev['eventId'],
            'minute': ev['minute'],
            'teamId': ev['teamId'],
            'playerId': player_id,
            'playerName': player_name,
            'matchId': matchId,
            'x': ev['x'],
            'y': ev['y'],
            'endX': endX,
            'endY': endY,
            'expandedMinute': ev['expandedMinute'],
            'period': ev['period']['displayName'],
            'type': ev['type']['displayName'],
            'outcomeType': ev['outcomeType']['displayName'],
            'isTouch': ev['isTouch']
        })
    event_df = pd.DataFrame(event_list)
    return event_df


In [ ]:
mev = get_match_events(driver,1640813)
# mev.to_csv('../../data/man_derby_touches.csv')